In [1]:
!pip install groq

In [2]:
!pip install pandas



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\sahil\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
filepath = "resumes_with_keywords_ipynb.csv"
def read_csv(file_path):
    df = pd.read_csv(file_path)  # Read CSV into DataFrame
    return df

In [4]:
import pandas as pd

df = pd.read_csv(filepath)
print(df.columns)  # Check the actual column names


Index(['Resume', 'Extracted Keywords'], dtype='object')


In [5]:
df.head()



,Resume,Extracted Keywords
0,Skills * Programming Languages: Python (pandas...,"fraud, topic, modelling, review, analytics, qu..."
1,Education Details \nMay 2013 to May 2017 B.E ...,"matelabs, outlier, year, scientist, exprience,..."
2,"Areas of Interest Deep Learning, Control Syste...","year, manipal, electrical, python, exprience, ..."
3,Skills â?¢ R â?¢ Python â?¢ SAP HANA â?¢ Table...,"sap, hana, analytics, industry, data, models, ..."
4,"Education Details \n MCA YMCAUST, Faridabad...","year, exprience, months, ymcaust, faridabad, i..."


In [7]:
df.tail()

,Resume,Extracted Keywords
957,Computer Skills: â?¢ Proficient in MS office (...,"resistance, test, breaker, insulation, circuit..."
958,â? Willingness to accept the challenges. â? ...,"electronics, quality, testing, abacus, instruc..."
959,"PERSONAL SKILLS â?¢ Quick learner, â?¢ Eagerne...","transformer, transformers, assembly, tests, in..."
960,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...,"pcb, matlab, matrix, project, mspower, protius..."
961,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,"testing, windows, windows7, xp, test, vista, a..."


In [8]:
df = pd.read_csv("resumes_with_keywords_ipynb.csv")
print(f"Total rows in CSV: {len(df)}")


Total rows in CSV: 962


In [32]:
import pandas as pd
from groq import Groq

filepath = "resumes_with_keywords_ipynb.csv"
def read_csv(file_path):
    df = pd.read_csv(file_path)  # Read CSV into DataFrame
    return df

def generate_questions_for_rows(file_path, output_file):
    df = read_csv(file_path)

    client = Groq(api_key="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")

    with open(output_file, "w", encoding="utf-8") as file:
        for index, row in df.iterrows():
            keywords = row['Extracted Keywords']

            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "Analyze the given data and generate only 1 interview questions for one keyword."
                    },
                    {
                        "role": "user",
                        "content": keywords  # Pass the keyword of the current row
                    }
                ],
                model="meta-llama/llama-4-scout-17b-16e-instruct",

            )

            questions = chat_completion.choices[0].message.content
            file.write(f"Questions for Row {index + 1} based on keywords:\n{questions}\n\n")
            print(f"Questions for Row {index + 1} saved.")

output_file = "generated_questions_ipynb.txt"
generate_questions_for_rows(filepath, output_file)
print(f"All questions have been saved in {output_file}.")

Questions for Row 1 saved.
Questions for Row 2 saved.
Questions for Row 3 saved.
Questions for Row 4 saved.
Questions for Row 5 saved.
Questions for Row 6 saved.
Questions for Row 7 saved.
Questions for Row 8 saved.
Questions for Row 9 saved.
Questions for Row 10 saved.
Questions for Row 11 saved.
Questions for Row 12 saved.
Questions for Row 13 saved.
Questions for Row 14 saved.
Questions for Row 15 saved.
Questions for Row 16 saved.
Questions for Row 17 saved.
Questions for Row 18 saved.
Questions for Row 19 saved.
Questions for Row 20 saved.
Questions for Row 21 saved.
Questions for Row 22 saved.
Questions for Row 23 saved.
Questions for Row 24 saved.
Questions for Row 25 saved.
Questions for Row 26 saved.
Questions for Row 27 saved.
Questions for Row 28 saved.
Questions for Row 29 saved.
Questions for Row 30 saved.
Questions for Row 31 saved.
Questions for Row 32 saved.
Questions for Row 33 saved.
Questions for Row 34 saved.
Questions for Row 35 saved.
Questions for Row 36 saved.
Q

KeyboardInterrupt: 

In [33]:
def check_keyword_relevance(keywords, question):
    found = sum(1 for kw in keywords if kw.lower() in question.lower())
    return found / len(keywords)


In [38]:
import re
import pandas as pd

# Read the text file
with open("generated_questions_ipynb.txt", "r", encoding="utf-8") as f:
    content = f.read()

# Extract keyword + question pairs
pattern = re.compile(
    r"Keywords?:\s*(.*?)\n.*?\*\*Interview Question:\*\*\s*(.*?)\n", 
    re.IGNORECASE | re.DOTALL
)

matches = pattern.findall(content)

# Store in structured format
data = []
for kw, question in matches:
    data.append({
        "Extracted Keywords": kw.strip(),
        "Generated Question": question.strip()
    })

df = pd.DataFrame(data)
df.to_csv("structured_generated_questions.csv", index=False)
print("✅ Converted to CSV: structured_generated_questions.csv")


✅ Converted to CSV: structured_generated_questions.csv


In [39]:
from groq import Groq
import pandas as pd
import textwrap

# Connect to Groq
client = Groq(api_key="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")

# Load structured question data
df = pd.read_csv("structured_generated_questions.csv")

# Combine all questions into one string
questions_block = ""
for index, row in df.iterrows():
    questions_block += f"{index+1}. {row['Generated Question']}\n"

# Prepare prompt
prompt = f"""
You are a strict and experienced technical interviewer.

Below is a list of interview questions generated by an AI system.

Evaluate the **overall quality** of these questions based on:
1. Relevance to their respective keywords
2. Clarity and grammar
3. Usefulness in a real interview

Here are the questions:
{textwrap.shorten(questions_block, width=10000, placeholder="...")}

Now rate the **overall quality** of this entire set on a scale from 0 (very poor) to 5 (excellent).
Only return a single number.
"""

# Send to Groq (LLaMA)
response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a strict evaluator of interview questions."},
        {"role": "user", "content": prompt}
    ],
    model="meta-llama/llama-4-scout-17b-16e-instruct",
)

# Extract final score
output = response.choices[0].message.content.strip()

import re
match = re.search(r"\b([0-5](?:\.\d)?)\b", output)
final_score = float(match.group(1)) if match else None

# Show result
print("\n📊 Overall Question Set Evaluation:")
print(f"🔹 LLaMA's Overall Quality Score: {final_score} / 5")



📊 Overall Question Set Evaluation:
🔹 LLaMA's Overall Quality Score: 3.0 / 5
